In [4]:
import numpy as np
import pandas as pd

In [5]:
from edge_detect import normalize, detect_lines

In [6]:
data_dir = "DAS_data"
files = os.listdir(data_dir)
print(len(files))

2334


In [54]:
labels = pd.read_excel("das_gopro_mapping_conf0.4.xlsx")
values = ['DAS_File', 'DAS_DateTime', 'DAS_Start_Seconds', 'DAS_End_Seconds',
       'GoPro_Start_Video', 'GoPro_Start_Time', 'GoPro_Start_Seconds',
       'GoPro_Start_Scaled_Time', 'GoPro_Start_Scaled_Seconds',
       'GoPro_End_Video', 'GoPro_End_Time', 'GoPro_End_Seconds',
       'GoPro_End_Scaled_Time', 'GoPro_End_Scaled_Seconds', 'class_0_count',
       'class_0_tracks', 'class_1_count', 'class_1_tracks', 'class_2_count',
       'class_2_tracks', 'class_3_count', 'class_3_tracks', 'class_5_count',
       'class_5_tracks', 'class_6_count', 'class_6_tracks', 'class_7_count',
       'class_7_tracks']
fill = {}
for c in values:
    if "count" in c:
        fill[c] = 0
    elif "tracks" in c:
        fill[c] = "[]"
    else:
        fill[c] = ""
labels = labels.fillna(value = fill)

In [55]:
labels.head(10)

,DAS_File,DAS_DateTime,DAS_Start_Seconds,DAS_End_Seconds,GoPro_Start_Video,GoPro_Start_Time,GoPro_Start_Seconds,GoPro_Start_Scaled_Time,GoPro_Start_Scaled_Seconds,GoPro_End_Video,...,class_2_count,class_2_tracks,class_3_count,class_3_tracks,class_5_count,class_5_tracks,class_6_count,class_6_tracks,class_7_count,class_7_tracks
0,sensor_2024-11-23T231921-0800.h5_Start0.0_End3...,2024-11-23T231921,0.0,30.0,GH010442,00:00,0.0,00:00,0.000000,GH010442,...,472.0,[1],0.0,[],0.0,[],0.0,[],0.0,[]
1,sensor_2024-11-23T231921-0800.h5_Start30.0_End...,2024-11-23T231921,30.0,60.0,GH010442,00:30,30.0,00:31,31.271416,GH010442,...,455.0,[1],0.0,[],0.0,[],0.0,[],0.0,[]
2,sensor_2024-11-23T232021-0800.h5_Start0.0_End3...,2024-11-23T232021,0.0,30.0,GH010442,01:00,60.0,01:02,62.542832,GH010442,...,410.0,[1],0.0,[],0.0,[],0.0,[],0.0,[]
3,sensor_2024-11-23T232021-0800.h5_Start30.0_End...,2024-11-23T232021,30.0,60.0,GH010442,01:30,90.0,01:33,93.814248,GH010442,...,390.0,[1],0.0,[],0.0,[],0.0,[],0.0,[]
4,sensor_2024-11-23T232121-0800.h5_Start0.0_End3...,2024-11-23T232121,0.0,30.0,GH010442,02:00,120.0,02:05,125.085664,GH010442,...,372.0,[1],0.0,[],0.0,[],0.0,[],0.0,[]
5,sensor_2024-11-23T232121-0800.h5_Start30.0_End...,2024-11-23T232121,30.0,60.0,GH010442,02:30,150.0,02:36,156.357080,GH010442,...,526.0,"[1, 26]",0.0,[],0.0,[],0.0,[],0.0,[]
6,sensor_2024-11-23T232221-0800.h5_Start0.0_End3...,2024-11-23T232221,0.0,30.0,GH010442,03:00,180.0,03:07,187.628496,GH010442,...,528.0,"[1, 37]",0.0,[],0.0,[],0.0,[],0.0,[]
7,sensor_2024-11-23T232221-0800.h5_Start30.0_End...,2024-11-23T232221,30.0,60.0,GH010442,03:30,210.0,03:38,218.899912,GH010442,...,400.0,"[1, 58]",0.0,[],0.0,[],0.0,[],0.0,[]
8,sensor_2024-11-23T232321-0800.h5_Start0.0_End3...,2024-11-23T232321,0.0,30.0,GH010442,04:00,240.0,04:10,250.171328,GH010442,...,455.0,"[58, 62]",0.0,[],0.0,[],0.0,[],0.0,[]
9,sensor_2024-11-23T232321-0800.h5_Start30.0_End...,2024-11-23T232321,30.0,60.0,GH010442,04:30,270.0,04:41,281.442744,GH010442,...,497.0,[62],0.0,[],0.0,[],0.0,[],0.0,[]


In [56]:
labels.columns

Index(['DAS_File', 'DAS_DateTime', 'DAS_Start_Seconds', 'DAS_End_Seconds',
       'GoPro_Start_Video', 'GoPro_Start_Time', 'GoPro_Start_Seconds',
       'GoPro_Start_Scaled_Time', 'GoPro_Start_Scaled_Seconds',
       'GoPro_End_Video', 'GoPro_End_Time', 'GoPro_End_Seconds',
       'GoPro_End_Scaled_Time', 'GoPro_End_Scaled_Seconds', 'class_0_count',
       'class_0_tracks', 'class_1_count', 'class_1_tracks', 'class_2_count',
       'class_2_tracks', 'class_3_count', 'class_3_tracks', 'class_5_count',
       'class_5_tracks', 'class_6_count', 'class_6_tracks', 'class_7_count',
       'class_7_tracks'],
      dtype='object')

In [57]:
def das_to_numpy(f):
    new_name = "DAS_data/FBE_sensor"
    f = f.replace("sensor", "")
    f = f.replace(".png", "")
    f = f.replace(".h5", "")
    f = f.replace("Start", "")
    f = f.replace("End", "")
    f = f.replace("0.0_30.0", "0s-30s")
    f = f.replace("30.0_60.0", "30s-60s")
    f = new_name + f + "_10-200Hz.npy"
    return f

In [64]:
from tqdm import tqdm
edge_labels = []
line_infos = []
for i, row in tqdm(labels.iterrows(), total = len(labels)):
    npy_path = das_to_numpy(row.DAS_File)
    np.load(npy_path)
    counts = 0
    for i in range(8):
        if i != 4:
            vehicles = row[f"class_{i}_tracks"]
            vehicles = [v for v in eval(str(row[f"class_{i}_tracks"])) if v != 1]
            counts += len(vehicles)
    if counts > 0:
        lines = detect_lines(npy_path, plot = False)
        line_infos.append(lines)
    else:
        line_infos.append([None])

100%|███████████████████████████████████████| 2334/2334 [01:19<00:00, 29.23it/s]


In [65]:
len(labels)

2334

In [66]:
len(line_infos)

2334

In [45]:
el = np.asarray(edge_labels)

In [77]:
line_infos

[[None],
 [None],
 [None],
 [None],
 [None],
 [],
 [],
 [(0, 0.5974496398351183, (382, -2), (539, 91))],
 [(2, -0.5303430624521364, (556, 30), (584, 15)),
  (10, -0.20409321088600269, (374, 104), (460, 86))],
 [(0, -0.31955587880991937, (2, 17), (22, 11)),
  (1, 0.6209744013212276, (401, 10), (414, 18))],
 [],
 [],
 [],
 [(0, 0.5094224761696321, (110, -6), (299, 89)),
  (3, 0.5947732552915674, (394, 134), (425, 152))],
 [(0, 0.8204751212122003, (126, -3), (249, 97)),
  (5, 0.37864928729323943, (304, 138), (327, 146))],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [(0, -0.29594647390326473, (555, 17), (581, 9))],
 [(3, -0.2591392502526429, (163, 22), (191, 14))],
 [(2, 0.6088653692067032, (0, 19), (117, 90)),
  (3, 0.7013657090917427, (138, 105), (162, 122)),
  (4, 0.547765974066552, (194, 141), (206, 148))],
 [],
 [(0, 0.2530041088456418, (140, 9), (149, 11)),
  (3, -0.4183238202329587, (87, 27), (114, 16))],
 [],
 [],
 [],
 [],
 [],
 [],
 [(0, -0.2160930973695331, (329, 18), (370, 10)),
  (2, 

In [47]:
np.unique(el, return_counts = True)

(array([-1,  0,  1]), array([1077,   68,  247]))

In [68]:
none = 0
unfound = 0
pos = 0
neg = 0
for line in line_infos:
    if line == []:
        unfound += 1
    elif line == [None]:
        none += 1
    else:
        for L in line:
            slope = L[1]
            if slope > 0:
                pos += 1
            else:
                neg += 1

In [79]:
len(labels) - none

1391

In [72]:
neg

293

In [73]:
none

943

In [74]:
unfound

481

In [75]:
pos

1551

In [76]:
neg

293